In [4]:
import pandas as pd
from tqdm import tqdm
import re
import os
import random
import nltk
import collections

random.seed(1991)

/home/prevot/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
SUMMRE_FOLDER = '/home/prevot/data/SUMMRE_DU_CAMPAIGN/new_du_csv/'

In [28]:
corpus_du = {}
inis = []
fins = []
for f in os.listdir(SUMMRE_FOLDER):
    tmp_df = pd.read_csv(SUMMRE_FOLDER+f)
    tmp_df = tmp_df.fillna('#')
    #print(tmp_df)
    session,speaker,annot = f.split('_')[0:3]
    tmp_dus = []
    cur_du = []
    for i,row in tmp_df.iterrows():
        if row['tok'] == '#' and cur_du !=[]:
            tmp_dus.append((cur_du[0],' '.join(cur_du[1])))
            if len(cur_du[1])>3:
                inis.append(cur_du[1][0])
                fins.append(cur_du[1][-1])
            cur_du = []
        else:
            if cur_du == []:
                cur_du = (row['start'],[str(row['tok'])])
            else:
                cur_du[1].append(str(row['tok']))
    if session in corpus_du.keys():
        corpus_du[session].update({speaker:tmp_dus})
    else:
        corpus_du[session] = {speaker:tmp_dus}

In [29]:
collections.Counter(fins)

Counter({'euh': 1906,
         'ça': 527,
         '@': 382,
         'hein': 376,
         'ouais': 340,
         'quoi': 291,
         'pas': 285,
         'de': 249,
         'là': 202,
         'aussi': 194,
         'oui': 190,
         "c'est": 173,
         'bien': 160,
         'que': 157,
         'mais': 155,
         'non': 130,
         'et': 127,
         'tout': 124,
         'voilà': 120,
         'faire': 117,
         'un': 107,
         'plus': 101,
         'même': 101,
         'sociaux': 100,
         'des': 96,
         'dire': 94,
         'le': 91,
         'du_coup': 91,
         'temps': 86,
         'en_fait': 85,
         'a': 81,
         'donc': 77,
         'vrai': 76,
         'bon': 75,
         'à': 75,
         'monde': 74,
         'pense': 73,
         'enfin': 71,
         "d'accord": 70,
         'ou': 68,
         'événement': 67,
         'fait': 66,
         'etcetera': 66,
         'qui': 64,
         'les': 63,
         'mal': 63,
         'o

In [30]:
collections.Counter(inis)

Counter({'et': 1325,
         'euh': 1279,
         'ouais': 913,
         'donc': 841,
         "c'est": 820,
         'mais': 745,
         'on': 727,
         'je': 716,
         'oui': 706,
         'ben': 680,
         'ah': 653,
         'non': 555,
         'ça': 420,
         '#': 312,
         'il': 270,
         'tu': 267,
         'pour': 263,
         'moi': 255,
         'qui': 252,
         'après': 241,
         'le': 240,
         'alors': 230,
         'un': 216,
         'de': 213,
         'ou': 202,
         'les': 200,
         'voilà': 194,
         'parce_que': 165,
         'si': 156,
         'en': 154,
         'des': 149,
         'j': 145,
         'la': 142,
         'à': 134,
         'ok': 127,
         '@': 125,
         'dans': 121,
         'enfin': 119,
         'bon': 114,
         'il_y': 109,
         "j'": 108,
         'hum': 100,
         'y': 95,
         'que': 93,
         'une': 93,
         'comme': 91,
         'pas': 90,
         'oh': 90

In [ ]:
# DM_SEM = donc?  (et), mais, alors, après, parce_que, du_coup
# DM_ATT = ah, ben, oh, enfin, bon 
# DM_feed = ouais, oui, mh, non, ok, voilà, d'accord,'peut-être', si
???? #

In [35]:
if 'summre' not in os.listdir('data_corpus/'):
    os.mkdir('data_corpus/summre/')

for conv in corpus_du.keys():
    tmp_sides = []
    for spk in corpus_du[conv].keys():
        tmp_df = pd.DataFrame(corpus_du[conv][spk],columns=['start','text'])
        tmp_df['spk'] = spk
        tmp_df['conv'] = conv
        tmp_sides.append(tmp_df)
    conv_df = pd.concat(tmp_sides)
    conv_df = conv_df.sort_values(by = 'start')
    conv_df.to_csv('data_corpus/summre/'+conv+'.csv',columns=['conv','spk','start','text'], encoding='utf-8', index=False)


# CREATE BENCHMARCK_BASE

In [6]:
SUMMRE_PATH = 'data_corpus/summre/'

In [19]:
def create_base(dmlist,base_name,context='all',n_max=100):
    '''
    IN : 
        dmlist : list of target dms
        base_name : sem or att (depending of the dm list provided, only for the output filename)
        context = all, dial(ogic), mono(logic)
        n_max : max number of example per dm
    '''
    candidates = {}
    for dm in dmlist:
        candidates[dm] = []

    for f in os.listdir(SUMMRE_PATH):
        conv = pd.read_csv(SUMMRE_PATH+f,names=['conv','spk','start','text'])
        conv['next_utt'] = conv['text'].shift(-1)
        conv['next_spk'] = conv['spk'].shift(-1)
        conv['next_start'] = conv['start'].shift(-1)
        for item,row in conv.iterrows():
            utt = row['text'].replace('#',"").replace('*','').replace('+',',').lower()    
            next_utt = str(row['next_utt']).replace('#',"").replace('*','').replace('+',',').lower()
            selected = False
            if (40 > len(utt.split())>3) and (40 >len(next_utt.split())>3):
                if context =='all':
                    selected = True 
                elif (row['spk']==row['next_spk']):
                    selected = (context =='mono')
                else:
                    selected = (context =='dial')
            if selected:
                first_tok = row['next_utt'].split()[0]
                for dm in candidates.keys():
                    if first_tok.lower() == dm:
                        candidates[dm].append(utt.lower()+'<$>'+dm+'<$>'+next_utt.lower())

    for dm in candidates.keys():
        print(dm+':'+str(len(candidates[dm])))

    all_candidates = []

    for c in candidates.keys():
        random.shuffle(candidates[c])
        all_candidates = all_candidates + candidates[c][0:n_max]

    random.shuffle(all_candidates)
    foutput = open('data_corpus/french_DM_'+base_name+'_'+context+'.txt','w')
    towrite_lines = '\n'.join(all_candidates)
    foutput.writelines(towrite_lines)
    foutput.close() 
    return 0

In [20]:
create_base(['donc', 'mais', 'alors', 'après', 'parce_que', 'ah', 'ben', 'oh', 'enfin', 'bon'],'all','all',50)


donc:348
mais:355
alors:98
après:106
parce_que:87
ah:363
ben:369
oh:53
enfin:59
bon:55


0

In [21]:
create_base(['donc', 'mais', 'parce_que', 'ben', 'enfin'],'all','mono',50)


donc:203
mais:114
parce_que:45
ben:69
enfin:42


0

In [22]:
create_base(['donc', 'mais', 'ah', 'ben'],'all','dial',125)


donc:145
mais:241
ah:333
ben:300


0

In [23]:
create_base(['donc', 'mais', 'alors', 'après'],'sem','all',100)


donc:348
mais:355
alors:98
après:106


0

In [27]:
create_base(['ah', 'ben'],'att','all',250)


ah:363
ben:369


0

In [28]:
create_base(['ah', 'ben'],'att','dial',250)


ah:333
ben:300


0

In [29]:
create_base([ 'ah', 'ben', 'oh', 'enfin', 'bon'],'att','mono',15)


ah:30
ben:69
oh:12
enfin:42
bon:16


0

# PROPOSAL for FRENCH DM TASKS 
     - ALL_ALL :  donc, mais, alors, après, parce_que, ah, ben, oh, enfin, bon (N = 10 x 50)
     - ALL_MONO : donc, mais, parce_que, ben, enfin (N = 5 x 50)
     - ALL_DIAL : donc, mais, ah, ben (N = 4 x 125)
     - ALL_SEM : donc, mais, alors, après (N = 4 x 100)
     - DIAL_SEM : donc, mais, alors, après (N = 4 x 75)
     - MONO_SEM : donc, mais (N = 2 x 100)
     - ALL_ATT : ah, ben (N= 2 x 250)
     - MONO_ATT : ah, ben, oh, enfin, bon (N = 5 x 15)
     - DIAL_ATT : ah, ben (N= 2 x 250)